In [1]:
from stexls.stex.compiler import StexObject, Compiler
from stexls.util.workspace import Workspace
from stexls.vscode import *

In [2]:
from pathlib import Path
from multiprocessing import Pool
from tqdm import tqdm

In [3]:
%cd ~/MathHub/

/home/marian/MathHub


In [4]:
root = Path('/home/marian/MathHub')
outdir = Path('/tmp/out')

ws = Workspace(root)
ws.include = '.*/TDM/source/.*'

In [5]:
c = Compiler(root, outdir)

In [6]:
args = list(zip(ws.files, map(lambda f: ws.read_file(f) if ws.is_open(f) else None, ws.files)))

In [9]:
objects = [c.compile(path, content=content) for path, content in tqdm(args)]

100%|██████████| 146/146 [00:05<00:00, 29.02it/s]


In [8]:
for o in objects[::20]:
    o.format()

Root: "/home/marian/MathHub"
File: "/home/marian/MathHub/MiKoMH/TDM/source/course/notes/toolsconcepts.tex"
Dependencies:n	No dependencies.
References:
	No references.
Errors:
	No errors.
Symbol Table:
> [public Symbol __root__]
-> [public Symbol __ANONYMOUS_SCOPE0__]
--> [public Symbol __ANONYMOUS_SCOPE1__]
Root: "/home/marian/MathHub"
File: "/home/marian/MathHub/MiKoMH/TDM/source/doccomp/fragments/genict.tex"
Dependencies:n	No dependencies.
References:
	No references.
Errors:
	No errors.
Symbol Table:
> [public Symbol __root__]
-> [public Symbol __ANONYMOUS_SCOPE35__]
Root: "/home/marian/MathHub"
File: "/home/marian/MathHub/MiKoMH/TDM/source/doccomp/en/regexpal.tex"
Dependencies:
	[Dependency at=[Range (1 3) (1 44)] module=regexp-ex type=ModuleType.MODULE file="/home/marian/MathHub/MiKoMH/TDM/source/doccomp/en/regexp-ex.tex" export=True]
References:
	[SymbolReference "['regexp-ex']" at [Range (1 3) (1 44)]]
	[SymbolReference "['regexp-practical']" at [Range (4 26) (4 42)]]
	[SymbolRef